**Objective -> To construct a program that:**
1. Take the input image from the user
2. Do operations to detect the Blobs in the given image
3. Show the Blobs as circles highlighted on the given image. 


**Operations to be implemented from scratch are:**
1. Build a Laplacian scale space (core function to be implemented on your own)
2. Filter image (core function to be implemented on your own, e.g, reuse the conv. in your hws)
3. Perform non-maximum suppression (core function to be implemented on your own)

In [23]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import time
import math
sigma_value=2
k_value=1.24

In [24]:
def hyperParameters():
    sigma=2
    k=1.24
    return sigma,k

In [33]:
def main():
    
    final_image,InputImage=getuserInput()
    print(final_image.shape)
    #save_blob=draw_circles(final_image,InputImage)
    #cv2.imwrite('NMSBlobsDected_sunflower.jpg',save_blob)
    cv2.imshow('FinalImage with Blobs',final_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #plt.imshow(final_image)
    #plt.imsave("BlobImage1.png",arr=final_image)    

In [26]:
def draw_circles(output_img,InputImage):
    rad = math.ceil(sigma_value*math.sqrt(2))
    data = 0
    for count in range(output_img.shape[2]):
        for i in range(InputImage.shape[0]):
            for j in range(InputImage.shape[1]):
                radius = rad*(k_value**(count+1))
                if(output_img[i][j][count]==1):
                    data = data+1
                    InputImage = cv2.circle(InputImage,(j,i),int(radius),(0,0,255))
    
    plt.imshow(InputImage)
    plt.imsave("BlobImage2.png",arr=InputImage)
    return InputImage

In [34]:
def getuserInput():
    location_of_img=input('Enter the filepath of the image (use forward slash only \"/"):')
    print()
    print("you've entered=",location_of_img)
    isLocCorrect=input("confirm with yes(y) or no(n) :")    
    print()
    InputImage=0
    #outputImg=0
    choose_method=" "
    if(isLocCorrect=='y'):
        proceed_flag=True
        choose_method=input("enter 1 for using increasing kernel size or enter 2 to use downsampling method:")
        print(type(choose_method))
    else:
        proceed_flag=False
        getuserInput()
    
    
    sigma,k = hyperParameters()   
    print(sigma,k)
    if(proceed_flag):
        InputImage=cv2.imread(location_of_img,0)
        InputImage_norm = np.array(InputImage,dtype=int)/255
        #plt.figure(1)
        #plt.imshow(InputImage)
        coloredImage= cv2.imread(location_of_img)
        #plt.figure(2)
        #plt.imshow(coloredImage)
        #cv2.imshow("inp",InputImage)
        #cv2.waitKey(5000)
        #cv2.destroyAllWindows()
        if(choose_method=='1'):
            print("reached if condt.")
            list_convolved = newScaleSpace(InputImage_norm,sigma,k)
            outputImg=nonMaxSup(list_convolved,sigma,k,InputImage_norm,coloredImage)
            ###outputImg=newScaleSpace(InputImage_norm,sigma,k)
        return outputImg,coloredImage
        
        

    

In [42]:
if __name__=='__main__':
    start_time = time.time()
    main()
    print("--- %s seconds ---" % (time.time() - start_time))
    

Enter the filepath of the image (use forward slash only "/"):D:/NCSU_PROJECTS/ECE558/TestImages4Project/fishes.jpg

you've entered= D:/NCSU_PROJECTS/ECE558/TestImages4Project/fishes.jpg
confirm with yes(y) or no(n) :n

Enter the filepath of the image (use forward slash only "/"):D:/NCSU_PROJECTS/ECE558/TestImages4Project/butterfly.jpg

you've entered= D:/NCSU_PROJECTS/ECE558/TestImages4Project/butterfly.jpg
confirm with yes(y) or no(n) :y

enter 1 for using increasing kernel size or enter 2 to use downsampling method:1
<class 'str'>
2 1.24
reached if condt.
creating a scale space
doing nonMax
575
2 1.24


TypeError: 'NoneType' object is not iterable

In [8]:
def logkernel(sigma,kernel_size):
    #kernel size =5x5 or 9x9 or X x X kernelSize = max(1,fix(6*sigma)+1)
    #use Vectorize to speed up
    #e=2.718
    #9=-4 to 4
    print("creating LoG kernel")
    #x=kernel_size
    #y=kernel_size
    center_px=int(((kernel_size-1)/2))
    #print("center",center_px)
    log_kernel=np.empty([kernel_size,kernel_size])
    #sum=0
    for i in range(kernel_size):
        for j in range(kernel_size):
            x=i-center_px
            y=j-center_px
            #log_kernel[i][j]=((-1.0)/(3.14*(sigma**4)))*(1-(((p)**2+(q)**2)/2*(sigma**2)))*(2.718** (-1*(p**2+q**2)/(2*(sigma**2))))
            ##log_kernel[i][j]=((x**2)+(y**2)-(2*(sigma**2))) * 2.718**(-1*(x**2+y**2)/(2*(sigma**2)))
            log_kernel[i][j] = ((-1.0)/(math.pi*(sigma**4)))*(1-((x**2+y**2)/(2*sigma**2)))*math.exp(-1*((x**2+y**2)/(2*sigma**2)))
            #sum=sum+log_kernel[i][j]
    #print(sum)
    return log_kernel


In [35]:
kernel_list=[]

def newScaleSpace(image,sigma,k):
    print("creating a scale space")
    scale_space=10
    list_images=[]
    output = np.zeros((image.shape[0],image.shape[1],scale_space))
    sigma_init=sigma
    
    #my_scale_space=()
    for i in range(scale_space):
        sigma_new=sigma_init*(k**i)
        #print(sigma_new)
        filter_size=(round(6*sigma_new))
        if(filter_size%2==0):
            filter_size=filter_size+1
        #print(filter_size)
        if(len(kernel_list)==scale_space):
            kernel=kernel_list[i]
        else:
            kernel=logkernel(sigma_new,filter_size)
            kernel_list.append(kernel) #method
        #print(kernel)
        kernel_normalized=np.array(kernel,dtype=float)*(sigma_new**2)
        img_im=convOp(image,kernel_normalized) #kernel
        #cv2.imshow(str(i),img_im)
        output[:,:,i]=img_im ###
   ###  outputImg=nonMaxSuppression(output,0.01) ###
        #print("iteration",i)
        list_images.append(img_im)
        
    #cv2.waitKey(0)
    
    
    #for i in range(2):
        #cv2.imshow(str(i),list_images[i])
    return  list_images #outputImg

In [41]:
def nonMaxSup(list_convoluted,sigma_value,k,InputImage,coloredImage):
    print("doing nonMax")
    sigma_init=sigma_value
    counter=0
    img=InputImage
    blobs_detected=coloredImage
    rad=sigma_init*np.sqrt(2)
    for layer in range(1,9):  #ignore first n last: if ScaleSpace=15 -> range(1,14)
        r=rad*k**(layer) #2,3,4 radius
        #sigma=sigma*k**(layer)
        for i in range(1,img.shape[0]-1):
            for j in range(1,img.shape[1]-1):
                px_value=list_convoluted[layer][i][j]
                #if px_value==0:
                #    continue
                flag=1
                for xx in range(-1,2):
                    for yy in range(-1,2):
                        for zz in range(-1,2):
                            if((xx!=0 or yy!=0 or zz!=0) and px_value<list_convoluted[layer+xx][i+yy][j+zz]):
                                flag=0
                                break
                if flag==1 and px_value>0.02:
                    cv2.circle(blobs_detected,(j,i),int(r),(0,0,255),1)
                    counter=counter+1
    print(counter)
    #cv2.imshow('nms_out',copy)
    #cv2.waitKey(10000)
    return blobs_detected

In [11]:
def isMax(arr,i,j,count,threshold):
    if(count<1):
        maxValue = np.amax(np.array(arr[i-1:i+2,j-1:j+2,count:count+2]))
    elif(count>noOfSamples-1):
        maxValue = np.amax(np.array(arr[i-1:i+2,j-1:j+2,count-1:count+1]))
    else:
        maxValue = np.amax(np.array(arr[i-1:i+2,j-1:j+2,count-1:count+2]))
    if(maxValue==0):
        return 0
    else:
        if((maxValue == arr[i][j][count]) & (maxValue>threshold)):
            return 1
        else:
            return 0

In [12]:
noOfSamples=10
def nonMaxSuppression(output,threshold):
    detector = np.zeros((output.shape[0],output.shape[1],output.shape[2]-2))
    maxPresent = 0
    for count in range(detector.shape[2]):
        for i in range(1,detector.shape[0]):
            for j in range(1,detector.shape[1]):
                detector[i][j][count] = isMax(output,i,j,count+1,threshold)
                if(detector[i][j][count]==1):
                    maxPresent = maxPresent+1
    print(maxPresent)
    return detector

In [127]:
abc=cv2.imread('D:/NCSU_PROJECTS/ECE558/TestImages4Project/butterfly.jpg',0)
abc_arr = np.array(abc,dtype=int)/255 #abc/np.max(abc)
ker=logkernel(2,13)
kernel_normalized=np.array(ker,dtype=float)*(2**2)
print(ker[6][6])
out_put=convOp(abc_arr,kernel_normalized)

creating LoG kernel
-8.0


In [14]:
def convOp(image, kernel):
    #print("doing convolution")
    ker_x=int(kernel.shape[0]/2) 
    ker_y=int(kernel.shape[1]/2)
    h,w=image.shape
    s=ker_x
    kernelSize=kernel.shape[0]
    #print(image.shape[0],image.shape[1])
    #create padding for obtaining same size Image
    paddedImage = (np.zeros((h+2*ker_x,w+2*ker_y)))
    paddedImage[ker_x:-ker_x,ker_y:-ker_y]=image
    conv2DImage=np.array(image)
    matrix=np.zeros(image.shape)
    #print(conv2DImage.shape)
    #print(paddedImage.shape)
    newImage_same = np.zeros(paddedImage.shape)
    for r in range(ker_x,paddedImage.shape[0]-ker_x):
        for c in range(ker_y,paddedImage.shape[1]-ker_x):
            matrix[r-ker_x,c-ker_y]=(np.sum(paddedImage[r-s:r+s+1,c-s:c+s+1]*kernel))**2
            
            #i=r-ker_x
            #j=c-ker_x
            #newImage_same[r][c]=(kernel*paddedImage[i:i+kernelSize,j:j+kernelSize]).sum()
            #output_image_same = newImage_same[ker_x:paddedImage.shape[0]-ker_x,ker_x:paddedImage.shape[1]-ker_x]
            #print(r,c)#conv2DImage
            #patch=image[r-:r+konstant_r,c:c+konstant_c]            
            #patch_product_pixels=np.multiply(patch,kernel)
            #patch_sum[r][c]=int(np.sum(patch_product_pixels))
    #maximum_px=np.max(matrix)
    #minimum_px=np.min(matrix)
    #for i in range(int(matrix.shape[0])):
    #    for j in range(int(matrix.shape[1])):
    #        conv2DImage[i][j]=(matrix[i][j]-minimum_px)*255.0/(maximum_px-minimum_px)
    #print(output_image_same.shape)
    #output = np.array(matrix,dtype=float)**2
    #matrix *= (255.0/matrix.max())
    #matrix=np.clip(matrix,0,255)
    return   matrix#conv2DImage # output  

In [128]:
cv2.imshow("aaa",out_put)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [123]:
from scipy.ndimage.filters import gaussian_laplace
g_l=gaussian_laplace(abc, sigma=3.9)**2 

In [124]:
cv2.imshow("aaa",g_l)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [131]:
print(logkernel(2,5))

creating LoG kernel
[[ 0.         -1.60588835 -2.42624842 -1.60588835  0.        ]
 [-1.60588835 -4.67292582 -6.17755838 -4.67292582 -1.60588835]
 [-2.42624842 -6.17755838 -8.         -6.17755838 -2.42624842]
 [-1.60588835 -4.67292582 -6.17755838 -4.67292582 -1.60588835]
 [ 0.         -1.60588835 -2.42624842 -1.60588835  0.        ]]


In [145]:
print(100*logkernel(1.4,5))

creating LoG kernel
[[ 1.12046117  0.63757111  0.06095148  0.63757111  1.12046117]
 [ 0.63757111 -2.4365444  -4.78239408 -2.4365444   0.63757111]
 [ 0.06095148 -4.78239408 -8.28586751 -4.78239408  0.06095148]
 [ 0.63757111 -2.4365444  -4.78239408 -2.4365444   0.63757111]
 [ 1.12046117  0.63757111  0.06095148  0.63757111  1.12046117]]
